In [4]:
# -*- coding: utf-8 -*-
"""
Created on Sat Oct  2 22:00:29 2021

@author:
"""
import pandas as pd
import requests
from bs4 import BeautifulSoup

import geocoder
from geopy import distance

from arcgis.gis import GIS
from arcgis.geocoding import geocode

from tqdm import tqdm_notebook as tqdm

def flat_info(flat):
    info = []
    try:
        flat_href = flat.find('div').find('a', class_='all-clickable').get('href')
        id_ = flat_href[-13:-5]
    except:
        id_ = None
        flat_href = None
    info.append(id_)
    info.append(flat_href)
    return info

def room_num(flat):
    #url = 'https://dom.ria.com'+url 
    try:
        numbers = flat.find('div', class_="mt-10 chars grey").find('span', class_="point-before").text.split(' ')[0]
    except:
        numbers = None
        print('room_num error')
    return numbers

def square(flat):
    try:
        square = flat.find('div', class_="mt-10 chars grey").find_all('span', class_="point-before")[-1].text.split(' ')[0]
    except:
        square = None
        print('square error')
    return square

def address(flat):
    try:
        address = flat.find('a', class_="all-clickable unlink").get('title')
    except:
        address = None
        print('address error')
    return address

def jk_ornot_jk(address):
    #повертає 1 якщо ЖК і 0, якщо не ЖК. На вхід дані з функції adress
    if 'ЖК' in address:
        return 1
    return 0

def area_score(flat):
    try:
        area = flat.find_all('a')[2].text
        score = df.loc[area[4:9],'Score']
    except:
        score = None
    return score

def price(flat):
    try:
        price = flat.find('div', class_='wrap_desc').find('b', class_='size18').text[:-3]
    except:
        price = None
    return price

def metro_dist(address):
    try:
        api_key = "AAPK3bc5da15c52a416c97064132b7d24dfb1TvLQpyo6kJZ6W3d90c1Y8tbiAryKLTLvf35AgZOeeqDINV0fX06CRqv_s8nCD_Z"
        portal = GIS("https://www.arcgis.com", api_key=api_key)
        geocode_results = geocode(address=address)
        adress_coord = []
        adress_coord.append(geocode_results[0].get('location').get('y'))
        adress_coord.append(geocode_results[0].get('location').get('x'))
        book = pd.read_excel(r'C:\Users\VladV\Documents\ОБУЧЕНИЕ\Metro_stations.xlsx')
        dist_to_metro = 1000
        for i in range(len(book)):
            metro_temp_coord = []
            metro_temp_coord = book.loc[i, 'Location']
            km_to_metro = 0
            km_to_metro = round(distance.distance(metro_temp_coord, adress_coord).km, 2)
            if km_to_metro < dist_to_metro:
                dist_to_metro = km_to_metro
    except:
        dist_to_metro = None
        print('metro_dist error')
    return dist_to_metro

'''не використовуємо цей метод пошуку відстані до метро
def metro(flat, flat_address):
    metro = flat.find_all('a')[-1].text.replace('"', '')
    flat_address = flat_address.replace(' ', '+' )
    #print(flat_adress)
    #print(metro)
    resp = requests.get(f'https://www.google.com/search?q=відстань+від+{flat_adress}+до+метро+{metro}', headers = HEADERS)
    soup = BeautifulSoup(resp.text, 'html.parser')
    try:
        dist = soup.find_all('span', class_='UdvAnf')[0].text.split(' ')[2][1:-2]
    except: 
        dist = None
    return dist'''

def time_to_metro(x):
    try:
        if x <= 1.0:
            x = 1
        elif 1.0 < x <= 3.0:
            x = 2
        else:
            x = 3
    except:
        x = None
        print('time_to_metro error')
    return x

def new_or_old(flat_href):
    try:
        url = 'https://dom.ria.com/v2'+flat_href
        resp = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(resp.text, 'html.parser')
        state = soup.find('div', class_='description mb-30').find('div', class_='size18').text.replace(' ', '')[1:-6]
        if state in 'Первинне житло':
            return 1
        return 0
    except:
        return None
        
    

df= pd.read_csv('Райони_Києва.csv')
df.rename(columns={'Назва Укр': 'Назва', 'Total':'Score'}, inplace=True)
df = df[['Назва', 'Score']]
df.set_index('Назва', inplace = True)
df.index = df.index.map(lambda x: x[:5])

adress_list = []
id_list = []
jk_list = []
area_score_list = []
room_num_list = []
square_list = []
metro_dist_list = []
metro_list2 = []
price_list = []
new_or_old_list = []
HEADERS = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
            }

start = 300
stop = 500
for page in tqdm(range(start, stop, 1)):
    url = f'https://dom.ria.com/uk/arenda-kvartir/kiev/?page={page}'
    resp = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(resp.text, 'html.parser')
    flats = soup.find_all('section', class_='realty-item')
    #print(flats[0])
    for flat in flats:
        addr = address(flat)
        id_list.append(flat_info(flat)[0])
        adress_list.append(addr)
        jk_list.append(jk_ornot_jk(addr))
        area_score_list.append(area_score(flat))
        room_num_list.append(room_num(flat))
        square_list.append(square(flat))
        metro_dist_list.append(metro_dist(addr))
        price_list.append(price(flat))
        new_or_old_list.append(new_or_old((flat_info(flat)[1])))


data = pd.DataFrame(id_list, columns =['sku'])
data['Price'] = price_list
data['Square'] = square_list
data['Rooms'], data['Area_score'], data['ЖК'] = room_num_list, area_score_list, jk_list
data['Первинне житло'] = new_or_old_list
data['Dist to metro'] = metro_dist_list
data['time_to_metro'] = data['Dist to metro'].apply(time_to_metro)
#    if i==0:
#        df = data
#    else:
#        df = pd.concat([df, data], ignore_index = True)
        



print (data)


<ipython-input-4-5230547b6578>:168: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page in tqdm(range(start, stop, 1)):


metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error
metro_dist error

           sku     Price Square Rooms  Area_score  ЖК  Первинне житло  \
0     21487104   34 574     171     3         4.0   0             0.0   
1     21504919   34 

In [5]:
data

,sku,Price,Square,Rooms,Area_score,ЖК,Первинне житло,Dist to metro,time_to_metro
0,21487104,34 574,171,3,4.0,0,0.0,0.62,1
1,21504919,34 574,83,2,4.0,0,0.0,6.50,3
2,21364053,42 553,65,2,3.0,1,0.0,0.41,1
3,21523113,42 553,140,3,3.0,0,0.0,0.46,1
4,21487246,53 191,78,2,4.0,0,0.0,0.77,1
...,...,...,...,...,...,...,...,...,...
3955,22195414,79 787,200,4,3.0,0,0.0,0.48,1
3956,22195413,119 681,135,3,4.0,0,0.0,0.63,1
3957,22195412,93 085,180,4,3.0,0,0.0,0.81,1
3958,22195411,47 872,120,3,3.0,0,0.0,0.69,1


In [6]:
data.to_excel(r"C:\Users\VladV\Documents\ОБУЧЕНИЕ\output_6.xlsx")